# Project 07 — Yield Curve Bootstrapping + Swap Pricing (Interactive)

## Goal
Build a **simple and educational** interest-rate curve workflow:

- Generate a set of **par swap rates** (synthetic or user-edited)
- **Bootstrap** discount factors \(P(0,T)\) and zero rates \(z(T)\)
- Price a **plain-vanilla fixed-for-floating swap**
- Visualize **curve shape**, **discount factors**, and **DV01 / sensitivity**

> Note: This notebook is intentionally "clean-room" and avoids day-count conventions, calendars, and multi-curve (OIS/IBOR) complexity.
It is meant to demonstrate the core ideas clearly.

In [1]:
from __future__ import annotations

import numpy as np
import pandas as pd
from pathlib import Path

import plotly.express as px
import plotly.graph_objects as go

import ipywidgets as widgets
from IPython.display import display

# --- Reproducibility ---
SEED = 42
rng = np.random.default_rng(SEED)

# --- Paths ---
PROJECT_DIR = Path.cwd()
ASSETS_DIR = PROJECT_DIR / "assets"
ASSETS_DIR.mkdir(parents=True, exist_ok=True)

print("CWD:", PROJECT_DIR)
print("ASSETS_DIR:", ASSETS_DIR.resolve())

CWD: c:\Users\Karim\Desktop\quant-finance-portfolio\projects
ASSETS_DIR: C:\Users\Karim\Desktop\quant-finance-portfolio\projects\assets


## 1) Curve inputs (par swap rates)

We assume annual fixed payments and annual floating resets for simplicity.

For a **par swap** with maturity \(N\) years and par rate \(S_N\):

\[
\text{PV(fixed)} = S_N \sum_{i=1}^{N} P(0,i)
\qquad
\text{PV(float)} = 1 - P(0,N)
\]

Par condition \(\text{PV(fixed)}=\text{PV(float)}\) gives:

\[
S_N \sum_{i=1}^{N} P(0,i) = 1 - P(0,N)
\]

Once \(P(0,1),\dots,P(0,N-1)\) are known, we solve:

\[
P(0,N) = \frac{1 - S_N \sum_{i=1}^{N-1} P(0,i)}{1 + S_N}
\]

In [2]:
def make_synthetic_par_swaps(max_maturity: int = 10, level: float = 0.03, slope: float = 0.002, curvature: float = -0.0001):
    """
    Synthetic par swap curve: S(T) = level + slope*(T-1) + curvature*(T-1)^2
    Rates are in decimal (0.03 = 3%).
    """
    T = np.arange(1, max_maturity + 1)
    S = level + slope * (T - 1) + curvature * (T - 1) ** 2
    S = np.maximum(S, 1e-4)  # keep positive for stability
    return pd.DataFrame({"Maturity": T, "ParSwapRate": S})

def bootstrap_discount_factors(par_swaps: pd.DataFrame, df_1y: float):
    """
    Bootstrap discount factors P(0,t) with annual payments.
    - Input par swaps with maturities 1..N
    - Provide P(0,1) as df_1y (from a deposit or simply assumed)
    """
    par_swaps = par_swaps.sort_values("Maturity").reset_index(drop=True)
    N = int(par_swaps["Maturity"].max())
    P = np.zeros(N + 1)  # P[0] unused, P[t] is DF at t years
    P[1] = df_1y

    for n in range(2, N + 1):
        S_n = float(par_swaps.loc[par_swaps["Maturity"] == n, "ParSwapRate"].iloc[0])
        sum_prev = P[1:n].sum()
        P[n] = (1.0 - S_n * sum_prev) / (1.0 + S_n)
        if P[n] <= 0:
            raise ValueError(f"Bootstrapping failed at maturity {n}: got DF={P[n]:.6f}. Try lower rates.")

    out = pd.DataFrame({
        "Maturity": np.arange(1, N + 1),
        "DF": P[1:]
    })
    out["ZeroRate_CC"] = -np.log(out["DF"]) / out["Maturity"]  # continuous-compounded zero
    out["ZeroRate_Simple"] = (1.0 / out["DF"] - 1.0) / out["Maturity"]
    return out

def price_swap_fixed_float(curve: pd.DataFrame, fixed_rate: float, maturity: int, notional: float = 1.0):
    """
    Price a payer/receiver fixed-for-floating swap under single-curve assumptions.
    Returns PV_fixed, PV_float, PV_swap (receiver fixed = PV_fixed - PV_float).
    """
    c = curve.set_index("Maturity")
    mats = np.arange(1, maturity + 1)
    dfs = c.loc[mats, "DF"].values

    pv_fixed = notional * fixed_rate * dfs.sum()
    pv_float = notional * (1.0 - dfs[-1])

    pv_receiver_fixed = pv_fixed - pv_float
    pv_payer_fixed = -pv_receiver_fixed
    return pv_fixed, pv_float, pv_receiver_fixed, pv_payer_fixed

def bump_curve(curve: pd.DataFrame, bump_bp: float = 1.0):
    """Parallel bump of zero rates (continuous compounding), then recompute DF."""
    bumped = curve.copy()
    bump = bump_bp * 1e-4
    z = bumped["ZeroRate_CC"].values + bump
    T = bumped["Maturity"].values
    bumped["DF"] = np.exp(-z * T)
    bumped["ZeroRate_CC"] = z
    return bumped

## 2) Interactive curve builder

- **Level**: shifts the curve up/down  
- **Slope**: steepens/flattens the curve  
- **Curvature**: adds "hump" or "smile" in rates  

We also need \(P(0,1)\) (1Y discount factor). For a 1Y simple rate \(r_{1y}\), a common approximation is:

\[
P(0,1)=\frac{1}{1+r_{1y}}
\]

In [3]:
level_w = widgets.FloatSlider(value=0.03, min=0.0, max=0.10, step=0.001, description="level", readout_format=".3f")
slope_w = widgets.FloatSlider(value=0.002, min=-0.01, max=0.01, step=0.0005, description="slope", readout_format=".4f")
curv_w  = widgets.FloatSlider(value=-0.0001, min=-0.002, max=0.002, step=0.0001, description="curvature", readout_format=".4f")

r1y_w   = widgets.FloatSlider(value=0.03, min=0.0, max=0.10, step=0.001, description="1Y rate", readout_format=".3f")
maxT_w  = widgets.IntSlider(value=10, min=3, max=30, step=1, description="max T")

swap_fixed_w = widgets.FloatSlider(value=0.035, min=0.0, max=0.15, step=0.0005, description="swap fixed", readout_format=".4f")
swap_mat_w   = widgets.IntSlider(value=5, min=1, max=30, step=1, description="swap mat")
notional_w   = widgets.FloatText(value=1_000_000, description="notional")

btn = widgets.Button(description="Build curve + price swap", button_style="success")
out = widgets.Output()

display(widgets.VBox([
    widgets.HTML("<b>Curve shape</b>"),
    widgets.HBox([level_w, slope_w, curv_w, maxT_w]),
    widgets.HTML("<b>1Y discount factor input</b>"),
    widgets.HBox([r1y_w]),
    widgets.HTML("<b>Swap to price</b>"),
    widgets.HBox([swap_fixed_w, swap_mat_w, notional_w]),
    btn,
    out
]))

In [4]:
def run_curve(_=None):
    with out:
        out.clear_output()

        par = make_synthetic_par_swaps(
            max_maturity=maxT_w.value,
            level=level_w.value,
            slope=slope_w.value,
            curvature=curv_w.value,
        )
        df1 = 1.0 / (1.0 + r1y_w.value)
        curve = bootstrap_discount_factors(par, df_1y=df1)

        # --- Plot: par swap rates ---
        fig_par = px.line(
            par, x="Maturity", y="ParSwapRate", markers=True, template="plotly_dark",
            title="Synthetic par swap curve"
        )
        fig_par.update_layout(yaxis_tickformat=".2%")
        fig_par.show()
        fig_par.write_html(ASSETS_DIR / "par_swap_curve.html")

        # --- Plot: zero rates ---
        fig_z = go.Figure()
        fig_z.add_trace(go.Scatter(x=curve["Maturity"], y=curve["ZeroRate_CC"], mode="lines+markers", name="Zero (cc)"))
        fig_z.update_layout(
            template="plotly_dark",
            title="Bootstrapped zero curve (continuous compounding)",
            xaxis_title="Maturity (years)",
            yaxis_title="Zero rate",
        )
        fig_z.update_yaxes(tickformat=".2%")
        fig_z.show()
        fig_z.write_html(ASSETS_DIR / "zero_curve.html")

        # --- Plot: discount factors ---
        fig_df = px.line(curve, x="Maturity", y="DF", markers=True, template="plotly_dark",
                         title="Discount factors P(0,T)")
        fig_df.update_layout(xaxis_title="Maturity (years)", yaxis_title="DF")
        fig_df.show()
        fig_df.write_html(ASSETS_DIR / "discount_factors.html")

        # --- Swap pricing ---
        maturity = int(swap_mat_w.value)
        maturity = min(maturity, int(curve["Maturity"].max()))
        fixed_rate = float(swap_fixed_w.value)
        notional = float(notional_w.value)

        pv_fixed, pv_float, pv_recv_fixed, pv_pay_fixed = price_swap_fixed_float(curve, fixed_rate, maturity, notional)

        # --- DV01 (parallel bump on zero rates) ---
        curve_up = bump_curve(curve, bump_bp=1.0)
        _, _, pv_recv_fixed_up, _ = price_swap_fixed_float(curve_up, fixed_rate, maturity, notional)
        dv01_recv_fixed = pv_recv_fixed_up - pv_recv_fixed  # price change for +1bp
        dv01_pay_fixed = -dv01_recv_fixed

        summary = pd.DataFrame({
            "Metric": [
                "PV fixed leg",
                "PV float leg",
                "PV swap (receiver fixed)",
                "PV swap (payer fixed)",
                "DV01 (receiver fixed, +1bp)",
                "DV01 (payer fixed, +1bp)",
            ],
            "Value": [pv_fixed, pv_float, pv_recv_fixed, pv_pay_fixed, dv01_recv_fixed, dv01_pay_fixed],
        })

        display(summary)

        display(curve.style.format({"DF":"{:.6f}", "ZeroRate_CC":"{:.4%}", "ZeroRate_Simple":"{:.4%}"}))

btn.on_click(run_curve)
run_curve()

## How to interpret the charts

- **Par swap curve**: market quotes for fixed rates that make swap PV = 0 at each maturity.  
- **Zero curve**: spot rates implied by no-arbitrage bootstrapping.
  - Upward slope often reflects **positive term premium** / inflation expectations.
- **Discount factors**: \(P(0,T)\) is the present value of 1 unit paid at time \(T\).
  - They decay with maturity; faster decay ↔ higher rates.

### DV01 intuition
DV01 is the **price sensitivity** to a 1bp parallel rate move.  
Receiver-fixed swap behaves like **long duration** (benefits when rates fall), so DV01(receiver) is typically negative under a +1bp bump.